**JOIN**

JOINs can be performed with join() or merge(). By default, join() will join the DataFrames on their indices. 

In [2]:
import numpy as np
import pandas as pd

In [3]:
df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})
df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

**INNER JOIN**

In [ ]:
SELECT *
FROM df1
INNER JOIN df2
  ON df1.key = df2.key;

In [4]:
# merge performs an INNER JOIN by default
pd.merge(df1, df2, on="key")

,key,value_x,value_y
0,B,-0.183833,0.780966
1,D,-2.487931,-0.874410
2,D,-2.487931,0.289457


**merge() also offers parameters for cases when you’d like to join one DataFrame’s column with another DataFrame’s index.**

In [5]:
indexed_df2 = df2.set_index("key")
pd.merge(df1, indexed_df2, left_on="key", right_index=True)

,key,value_x,value_y
1,B,-0.183833,0.780966
3,D,-2.487931,-0.874410
3,D,-2.487931,0.289457


**LEFT OUTER JOIN**

In [ ]:
#show all records from df1
SELECT *
FROM df1
LEFT OUTER JOIN df2
  ON df1.key = df2.key;

In [6]:
# show all records from df1
pd.merge(df1, df2, on="key", how="left")

,key,value_x,value_y
0,A,0.667794,NaN
1,B,-0.183833,0.780966
2,C,0.252920,NaN
3,D,-2.487931,-0.874410
4,D,-2.487931,0.289457


**RIGHT JOIN**

In [ ]:
 show all records from df2
SELECT *
FROM df1
RIGHT OUTER JOIN df2
  ON df1.key = df2.key;


In [9]:
# show all records from df2
pd.merge(df1, df2, on="key", how="right")

,key,value_x,value_y
0,B,-0.183833,0.780966
1,D,-2.487931,-0.874410
2,D,-2.487931,0.289457
3,E,NaN,0.745502


**FULL JOIN**

pandas also allows for FULL JOINs, which display both sides of the dataset, whether or not the joined columns find a match. As of writing, FULL JOINs are not supported in all RDBMS (MySQL).


In [ ]:
#show all records from both tables
SELECT *
FROM df1
FULL OUTER JOIN df2
  ON df1.key = df2.key;

In [10]:
# show all records from both frames
pd.merge(df1, df2, on="key", how="outer")

,key,value_x,value_y
0,A,0.667794,NaN
1,B,-0.183833,0.780966
2,C,0.252920,NaN
3,D,-2.487931,-0.874410
4,D,-2.487931,0.289457
5,E,NaN,0.745502


**UNION**

UNION ALL can be performed using concat().


In [12]:
df1 = pd.DataFrame(
     {"city": ["Chicago", "San Francisco", "New York City"], "rank": range(1, 4)}
   )

In [13]:
df2 = pd.DataFrame(
{"city": ["Chicago", "Boston", "Los Angeles"], "rank": [1, 4, 5]})

In [ ]:
SELECT city, rank
FROM df1
UNION ALL
SELECT city, rank
FROM df2;
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
      Chicago     1
       Boston     4
  Los Angeles     5
*/

In [14]:
pd.concat([df1, df2])

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
0,Chicago,1
1,Boston,4
2,Los Angeles,5


**SQL’s UNION is similar to UNION ALL, however UNION will remove duplicate rows.**

In [ ]:
SELECT city, rank
FROM df1
UNION
SELECT city, rank
FROM df2;
-- notice that there is only one Chicago record this time
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
       Boston     4
  Los Angeles     5
*/

**In pandas, you can use concat() in conjunction with drop_duplicates().**

In [15]:
pd.concat([df1, df2]).drop_duplicates()

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
1,Boston,4
2,Los Angeles,5


**pandas equivalents for some SQL analytic and aggregate functions**

In [ ]:
#Top n rows with offset
-- MySQL
SELECT * FROM tips
ORDER BY tip DESC
LIMIT 10 OFFSET 5;

In [19]:
url = (
   ...:     "https://raw.github.com/pandas-dev"
   ...:     "/pandas/master/pandas/tests/io/data/csv/tips.csv"
   ...: )
   ...: 

tips = pd.read_csv(url)

tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [20]:
tips.nlargest(10 + 5, columns="tip").tail(10)

,total_bill,tip,sex,smoker,day,time,size
183,23.17,6.50,Male,Yes,Sun,Dinner,4
214,28.17,6.50,Female,Yes,Sat,Dinner,3
47,32.40,6.00,Male,No,Sun,Dinner,4
239,29.03,5.92,Male,No,Sat,Dinner,3
88,24.71,5.85,Male,No,Thur,Lunch,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2
44,30.40,5.60,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
85,34.83,5.17,Female,No,Thur,Lunch,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4


**Top n rows per group**

In [ ]:
 Oracle's ROW_NUMBER() analytic function
SELECT * FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER(PARTITION BY day ORDER BY total_bill DESC) AS rn
  FROM tips t
)
WHERE rn < 3
ORDER BY day, rn;

In [21]:
(
tips.assign(
    rn=tips.sort_values(["total_bill"], ascending=False)
    .groupby(["day"])
    .cumcount()
    + 1
      )
      .query("rn < 3")
      .sort_values(["day", "rn"])
    )
   

,total_bill,tip,sex,smoker,day,time,size,rn
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1
212,48.33,9.00,Male,No,Sat,Dinner,4,2
156,48.17,5.00,Male,No,Sun,Dinner,6,1
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1
142,41.19,5.00,Male,No,Thur,Lunch,5,2


**The same using rank(method='first') function**

In [22]:
(
  tips.assign(
      rnk=tips.groupby(["day"])["total_bill"].rank(
      method="first", ascending=False
            )
      )
   .query("rnk < 3")
     .sort_values(["day", "rnk"])
   )

,total_bill,tip,sex,smoker,day,time,size,rnk
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1.0
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2.0
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1.0
212,48.33,9.00,Male,No,Sat,Dinner,4,2.0
156,48.17,5.00,Male,No,Sun,Dinner,6,1.0
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2.0
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1.0
142,41.19,5.00,Male,No,Thur,Lunch,5,2.0


**Let’s find tips with (rank < 3) per gender group for (tips < 2). Notice that when using rank(method='min') function rnk_min remains the same for the same tip (as Oracle’s RANK() function)**

In [ ]:
#Oracle's RANK() analytic function
SELECT * FROM (
  SELECT
    t.*,
    RANK() OVER(PARTITION BY sex ORDER BY tip) AS rnk
  FROM tips t
  WHERE tip < 2
)
WHERE rnk < 3
ORDER BY sex, rnk;

In [23]:
(
  tips[tips["tip"] < 2]
  .assign(rnk_min=tips.groupby(["sex"])["tip"].rank(method="min"))
  .query("rnk_min < 3")
 .sort_values(["sex", "rnk_min"])
)
      

,total_bill,tip,sex,smoker,day,time,size,rnk_min
67,3.07,1.00,Female,Yes,Sat,Dinner,1,1.0
92,5.75,1.00,Female,Yes,Fri,Dinner,2,1.0
111,7.25,1.00,Female,No,Sat,Dinner,1,1.0
236,12.60,1.00,Male,Yes,Sat,Dinner,2,1.0
237,32.83,1.17,Male,Yes,Sat,Dinner,2,2.0


**UPDATE**

In [ ]:
UPDATE tips
SET tip = tip*2
WHERE tip < 2;

In [25]:
tips.loc[tips["tip"] < 2, "tip"] *= 2

**DELETE**

In [ ]:
DELETE FROM tips
WHERE tip > 9;

In [26]:
#In pandas we select the rows that should remain, instead of deleting 
tips = tips.loc[tips["tip"] <= 9]